In [ ]:
import random
from pprint import pprint

import ipywidgets as widgets
import pandas as pd
import torch
from IPython.display import Audio, clear_output, display
from transformers import logging

from unified_desktop import RESOURCES_DIR
from unified_desktop.core.utils.io_utils import get_matching_files_in_dir
from unified_desktop.pipelines import (
    UDIntentClassifier,
    UDKeyExtraction,
    UDSentimentDetector,
    UDSpeechEmotionRecognizer,
    UDSpeechRecognizer,
    UDSummarizer,
)

pd.options.display.max_colwidth = 100
%load_ext autoreload
%autoreload 2
clear_output()

In [ ]:
# Cuda dropdown
CUDA_OPTIONS = [torch.device("cuda", idx) for idx in range(torch.cuda.device_count())]
device_dropdown = widgets.Dropdown(
    options=["cpu"] + CUDA_OPTIONS,
    value=torch.device("cuda", 0) if torch.cuda.is_available() else "cpu",
    description="Device:",
)
display(device_dropdown)

## Automatic Speech Recognition (ASR)

In [ ]:
def update_asr_obj(change):
    global asrObj
    clear_output()
    display(model_dropdown_ASR, device_dropdown)
    asrObj = UDSpeechRecognizer(model_dropdown_ASR.value, device=device_dropdown.value)


# Dropdown for OpenAI Whisper models
model_dropdown_ASR = widgets.Dropdown(
    options=UDSpeechRecognizer.available_models,
    value="distil-whisper/distil-medium.en",
    description="ModelName:",
)

# Attach the update function to the dropdown
model_dropdown_ASR.observe(update_asr_obj, names="value")
device_dropdown.observe(update_asr_obj, names="value")

# Display the widgets and initialize asrObj
display(model_dropdown_ASR)
asrObj = UDSpeechRecognizer(model_dropdown_ASR.value, device=device_dropdown.value)

In [ ]:
# Audio file to transcribe
audio_file = (
    RESOURCES_DIR
    / "call-center-sample-en_US"
    / "en_US_7a4f56d7-9aca-4ed5-96b9-9c9c36b8a3ac.wav"
)
audio = Audio(filename=audio_file)
display(audio)

# Transcribe the audio file
transcribed_text = asrObj(audio_file)
pprint(transcribed_text)

## Speech Emotion Recognition (SER)

In [ ]:
def update_ser_obj(change):
    global serObj
    serObj = UDSpeechEmotionRecognizer(device=device_dropdown.value)
    clear_output()
    display(device_dropdown)
    print(f"Loaded SER model on device: {device_dropdown.value}")


device_dropdown = widgets.Dropdown(
    options=["cpu"] + CUDA_OPTIONS,
    value="cpu",
    description="Device:",
)

# Attach the update function to the dropdown
device_dropdown.observe(update_ser_obj, names="value")

# Display the widgets and initialize serObj
serObj = UDSpeechEmotionRecognizer(device=device_dropdown.value)
clear_output()
display(device_dropdown)

In [ ]:
# Recognize the emotion in the audio file
emotion_pred_per_audio = {}
for audio_file in get_matching_files_in_dir(
    RESOURCES_DIR / "emotions-sample-en", "*.wav"
):
    emotion_pred_per_audio[audio_file] = serObj(audio_file)
    clear_output()

# Display the audio file and the recognized emotion
for audio_file, emotion_pred in emotion_pred_per_audio.items():
    audio = Audio(filename=audio_file)
    display(audio)
    print(f"Emotion: {emotion_pred.label}, Score: {emotion_pred.score:.3f}")

# Intent Detection

In [ ]:
def update_intent_obj(change):
    global intentObj
    clear_output()
    display(model_dropdown_intent, device_dropdown)
    intentObj = UDIntentClassifier(
        name=model_dropdown_intent.value, device=device_dropdown.value
    )
    print(
        f"Loaded model: {model_dropdown_intent.value} on device: {device_dropdown.value}"
    )


# Dropdown for intentDetection transformer models. More models will be added
model_dropdown_intent = widgets.Dropdown(
    options=UDIntentClassifier.available_models,
    value="vineetsharma/customer-support-intent-albert",
    description="ModelName:",
)

# Attach the update function to the dropdown
model_dropdown_intent.observe(update_intent_obj, names="value")
device_dropdown.observe(update_intent_obj, names="value")

# Display the widgets and initialize intentObj
display(model_dropdown_intent)
intentObj = UDIntentClassifier(
    name=model_dropdown_intent.value,
    device=device_dropdown.value,
)

In [ ]:
# Input text for intent detection
excel_file = RESOURCES_DIR / "input_text" / "textqueries.xlsx"

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file)

# Specify the row number (k) you want to select (replace with the desired row number)
num_rows = df.shape[0]
random_row_index = random.randint(0, num_rows - 1)
selected_row = df.iloc[random_row_index]

# Convert the selected row to a string
intent_input = selected_row.to_string(index=False)

# Display the selected row as a string
print(f"inpute query:")
print(intent_input, "\n")

# returns top_k predictions
top_k = 2

# Perform intent detection
intent_results = intentObj(intent_input, top_k)

# Print the top-k predictions and their probabilities for the intent detection
print(f"Top-{top_k} Intent Predictions:")
print("------------------------------------------------------")
print("| Intent                    | Probability")
print("------------------------------------------------------")
for items in intent_results:
    print(f"| {items['label']:<25} | {items['score']:.4f}")
print("------------------------------------------------------")

# Keyword Detection

In [ ]:
# available_models are the list of all transformer models
# that works well for the Keyword Extraction purpose.
# More will be added to the list after testing each one.

available_models = ["yanekyuk/bert-uncased-keyword-extractor"]


def update_key_obj(change):
    global KeyObj
    clear_output()
    display(model_dropdown_key, device_dropdown)
    KeyObj = UDKeyExtraction(
        available_models,
        name=model_dropdown_key.value,
        device=device_dropdown.value,
    )
    print(
        f"Loaded model: {model_dropdown_key.value} on device: {device_dropdown.value}"
    )


# Dropdown for UDKeyExtraction transformer models. More models will be added
model_dropdown_key = widgets.Dropdown(
    options=available_models,
    value="yanekyuk/bert-uncased-keyword-extractor",
    description="ModelName:",
)

# Attach the update function to the dropdown
model_dropdown_key.observe(update_key_obj, names="value")
device_dropdown.observe(update_key_obj, names="value")

# Display the widgets and initialize keyObj
display(model_dropdown_key)
KeyObj = UDKeyExtraction(name=model_dropdown_key.value, device=device_dropdown.value)

In [ ]:
# Input text for keywords extraction
content_input = """Googleeeeee is being investigated by the UK antitrust watchdog for 
its dominance in the 'ad tech stack,' the set of services that facilitate the sale of 
online advertising space between advertisers and sellers. Googleeeeee has strong positions 
at various levels of the ad tech stack and charges fees to both publishers and advertisers. 
A step back: UK Competition and Markets Authority has also been investigating whether Google 
and Meta colluded over ads, probing into the advertising agreement between the two companies, 
codenamed Jedi Blueeeeeee."""

# Display the selected row as a string
print(f"Input Query:")
print(content_input, "\n")

# Perform keywords extraction
key_results = KeyObj(content_input)
print("keywords extraction:")
print("------------------------------------------------------")
print("| index   | keyword               | Probability")
print("------------------------------------------------------")

for item in key_results:
    print(f"| {item['index']:<5} | {item['word']:<23} | {item['score']:.4f}")

print("------------------------------------------------------")

## Transcription Summarization

In [ ]:
summarizer = UDSummarizer(device=device_dropdown.value)
summary = summarizer(transcribed_text, max_length=50, min_length=30)
print(summary)

## Sentiment Detection

In [ ]:
sentiment_obj = UDSentimentDetector()
text_inputs = [
    "Thank you for your support",
    "Thank you for going above and beyond to address my issues",
    "I am calling regarding my prescriptions",
    "I have been waiting in the phone line for 20 minutes",
    "I am very dissapointed with this type of service",
]
output_sentiment = [sentiment_obj(input_text=text_ele) for text_ele in text_inputs]
print(*output_sentiment, sep="\n")